## Functions for drawing

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [6]:
def average_keypoints(keypoints, confidence_threshold, frame):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    total_considered = 0
    total_y = 0
    total_x = 0
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            total_considered += 1
            total_y += ky
            total_x += kx
    
    average_y = total_y / total_considered
    average_x = total_x / total_considered
    
    return average_x, average_y

In [7]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [11]:
PISTE_EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'c',
    (2, 3): 'm',
}

In [12]:
def draw_pistebox(frame, keypoints, edges):
    for edge, color in edges.items():
        p1, p2 = edge
        x1, y1 = keypoints[p1]
        x2, y2 = keypoints[p2]    
        cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [16]:
import math

def distCalc(pt1, pt2):
    dis = math.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2)
    return dis

In [22]:
leftEnGardeToMid = 200 / distCalc(pisteLines['middleLines'], pisteLines['leftEngardeLine'])
rightEnGardeToMid = 200 / distCalc(pisteLines['middleLines'], pisteLines['rightEngardeLine'])
leftEngardeToLeftWarning = 300 / distCalc(pisteLines['leftEngardeLine'], pisteLines['leftWarningLine'])
rightEngardeToRightWarning = 300 / distCalc(pisteLines['rightEngardeLine'], pisteLines['rightWarningLine'])
print(leftEnGardeToMid, rightEnGardeToMid, leftEngardeToLeftWarning, rightEngardeToRightWarning)

0.7008855588796173 0.8054961244402359 0.5879604230113189 0.911284943730949


In [25]:
def calc_dist_from_4m(fencer):
    # Negative if to the left of engarde, positive otherwise
    if (fencer[0] < pisteLines['leftEngardeLine'][0]):
        # Left side
        distFromLeftEGLine = leftEngardeToLeftWarning * (fencer[0] - pisteLines['leftEngardeLine'][0])
        return distFromLeftEGLine - 200
    elif (fencer[0] < pisteLines['middleLines'][0]):
        distFromMidLine = leftEnGardeToMid * (fencer[0] - pisteLines['middleLines'][0])
        return distFromMidLine
    elif (fencer[0] < pisteLines['rightEngardeLine'][0]):
        distFromMidLine = rightEnGardeToMid * (fencer[0] - pisteLines['middleLines'][0])
        return distFromMidLine
    else:
        distFromRightEGLine = rightEngardeToRightWarning * (fencer[0] - pisteLines['rightEngardeLine'][0])
        return 200 + distFromRightEGLine
        

In [29]:
left_dist = []
for coord in left_fencer:
    left_dist.append(calc_dist_from_4m(coord))
    
right_dist = []
for coord in right_fencer:
    right_dist.append(calc_dist_from_4m(coord))

In [33]:
dist_bw_fencers = []
for i in range(len(left_dist)):
    dist_bw_fencers.append(((right_dist[i] + 700) - (left_dist[i] + 700)))

In [35]:
# Velocity of fencers 
vel_left = []
for i in range(len(left_dist) - 1):
    vel_left.append((left_dist[i + 1] - left_dist[i]))

## Setting up AlphaPose

In [1]:
! pip install pyyaml==5.2
! pip install scipy
! pip install numpy==1.23.5
! pip install opencv-python
! pip install pillow
! pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117
! pip install cython

     ---------------------------------------- 0.0/265.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/265.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/265.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/265.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/265.7 kB ? eta -:--:--
     - -------------------------------------- 10.2/265.7 kB ? eta -:--:--
     ---- -------------------------------- 30.7/265.7 kB 100.4 kB/s eta 0:00:03
     ----- ------------------------------- 41.0/265.7 kB 115.5 kB/s eta 0:00:02
     ----- ------------------------------- 41.0/265.7 kB 115.5 kB/s eta 0:00:02
     -------- ---------------------------- 61.4/265.7 kB 142.2 kB/s eta 0:00:02
     ----------- ------------------------- 81.9/265.7 kB 169.9 kB/s eta 0:00:02
     ------------ ------------------------ 92.2/265.7 kB 180.4 kB/s eta 0:00:01
     --------------- -------------------- 112.6/265.7 kB 204.8 kB/s eta 0:00:

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\admin\\anaconda3\\Lib\\site-packages\\~aml\\_yaml.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



^C
^C
^C



KeyboardInterrupt



In [1]:
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

Cloning into 'AlphaPose'...


In [3]:
os.chdir('..')

In [7]:
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

  Cloning https://github.com/philferriere/cocoapi.git to c:\users\admin\appdata\local\temp\pip-install-qnkih8lw\pycocotools_12bcd66171a64605b1499186be95fb20
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pycocotools: filename=pycocotools-2.0-cp311-cp311-win_amd64.whl size=83761 sha256=b7c3e3221cb3ac663ff06aeb141c459f3b4240163d1660a82efd4cb050bd3543
  Stored in directory: C:\Users\admin\AppData\Local\Temp\pip-ephem-wheel-cache-b2d6zsz9\wheels\48\26\68\d7169b79632db4cf9ce71d33befbf0d89ae8b2d4d445ee8574
Successfully built pycocotools


  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git 'C:\Users\admin\AppData\Local\Temp\pip-install-qnkih8lw\pycocotools_12bcd66171a64605b1499186be95fb20'


In [74]:
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

  Cloning https://github.com/philferriere/cocoapi.git to c:\users\nicho\appdata\local\temp\pip-req-build-x3fputrc
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


DEPRECATION: Loading egg at c:\users\nicho\anaconda3\envs\fyp\lib\site-packages\munkres-1.1.4-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\nicho\anaconda3\envs\fyp\lib\site-packages\natsort-8.4.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\nicho\anaconda3\envs\fyp\lib\site-packages\timm-0.1.20-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git 'C:\Users\nicho\AppData\Local\Temp\pip-req-build-x3fputrc'


In [8]:
!python -m pip install git+https://github.com/yanfengliu/cython_bbox.git

  Cloning https://github.com/yanfengliu/cython_bbox.git to c:\users\admin\appdata\local\temp\pip-req-build-nca4vea1
  Resolved https://github.com/yanfengliu/cython_bbox.git to commit e7b95b6dc0933b10bd0ec02b90755a12ca996c68
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp311-cp311-win_amd64.whl size=29324 sha256=9dd49f3feaeb7399b0fec8b9b170b29de8053a539ae7b50f5373abf42a391ba2
  Stored in directory: C:\Users\admin\AppData\Local\Temp\pip-ephem-wheel-cache-g_hzpjiv\wheels\07\90\06\a1d2be022e17f568bd1d57ba8d91cd2ee97c46c3960866e05a
Successfully built cython-bbox


  Running command git clone --filter=blob:none --quiet https://github.com/yanfengliu/cython_bbox.git 'C:\Users\admin\AppData\Local\Temp\pip-req-build-nca4vea1'


In [6]:
%run setup.py build develop

Compiling detector\nms\src/soft_nms_cpu.pyx because it depends on C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\Cython\Includes\libc\string.pxd.
[1/1] Cythonizing detector\nms\src/soft_nms_cpu.pyx


C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
2023-09-07 01:40:13 [INFO]: running build
2023-09-07 01:40:13 [INFO]: running build_py
2023-09-07 01:40:13 [INFO]: copying alphapose\version.py -> build\lib.win-amd64-cpython-311\alphapose
2023-09-07 01:40:14 [INFO]: running build_ext
C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torch\utils\cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnin

AttributeError: 'tuple' object has no attribute 'tb_frame'

## Alpha Pose running

In [2]:
import torch
print(torch.__version__)
import yaml, scipy, os
print(yaml.__version__)
print(scipy.__version__)

2.0.1+cu117
5.2
1.11.2


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
import os
os.chdir('./AlphaPose')

In [49]:
import cv2
import numpy as np

cap = cv2.VideoCapture("samplefencing.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

ret, frame = cap.read()
refPt = []
cropping = False

clone = frame.copy()
def click_and_crop(event, x, y, flags, param):
    # grab references to the global variables
    global refPt, cropping
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being
    # performed
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True
    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        refPt.append((x, y))
        cropping = False
        # draw a rectangle around the region of interest
        cv2.rectangle(frame, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", frame)

cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)
# keep looping until the 'q' key is pressed
while True:
    # display the image and wait for a keypress
    cv2.imshow("image", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the 'r' key is pressed, reset the cropping region
    if key == ord("r"):
        frame = clone.copy()
    # if the 'c' key is pressed, break from the loop
    elif key == ord("q"):
        break

# create a mask
mask = np.zeros(frame.shape[:2], np.uint8)
mask[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]] = 255

# Apply mask
frame = cv2.bitwise_and(frame,frame,mask=mask)

# Perform cropping
# frame = frame[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
# res = cv2.resize(frame,None,fx=4, fy=4, interpolation = cv2.INTER_CUBIC)
# print(res.shape)

# Write the frame into the file 'output.avi'
# out = cv2.VideoWriter('output.mp4',fourcc, 30, (res.shape[1],res.shape[0]))
out = cv2.VideoWriter('masked.mp4',fourcc, 30, (frame_width,frame_height))
out.write(frame)
        

while(True):
    ret, frame = cap.read()
 
    if ret == True: 
        # Apply mask
        frame = cv2.bitwise_and(frame,frame,mask=mask)
#         frame = frame[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
#         res = cv2.resize(frame,None,fx=4, fy=4, interpolation = cv2.INTER_CUBIC)
        
        # Write the frame into the file 'output.avi'
        out.write(frame)
        
        # Display the resulting frame    
        cv2.imshow('frame',frame)

        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print(frame_width)
        break
 

out.release()
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

1920


In [46]:
out.release()
cap.release()
cv2.destroyAllWindows()

In [2]:
%run scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --video ../samplefencing.mp4 --outdir ../output/ --qsize 10 --detbatch 3 --posebatch 30 --gpus 0 --save_video --pose_track --vis_fast --sp --debug

Exception: File `'scripts/demo_inference.py'` not found.

## Fast Pose DUC Unshuffle MSCOCO 

In [5]:
%run scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x-duc.yaml --checkpoint pretrained_models/fast_421_res50-shuffle_256x192.pth --video ../output.mp4 --outdir ../output/ --qsize 50 --detbatch 5 --posebatch 50 --gpus 0 --pose_track --save_video --showbox --vis_fast --debug

2023-10-07 21:47:47 [DEBUG]: Loaded backend agg version v2.2.


Load shuffle backbone...
Loading YOLO model..


C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pose model from pretrained_models/fast_421_res50-shuffle_256x192.pth...
[0]
cuda:0
loading reid model from trackers/weights/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth...


  0%|          | 0/15451 [00:00<?, ?it/s]C:\Users\admin\Documents\Anaconda Projects\AlphaPose\trackers\__init__.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  new_hm = torch.Tensor(new_hm).to(args.device)
100%|██████████| 15451/15451 [21:58<00:00, 11.72it/s] 


===========================> Finish Model Running.
Results have been written to json.


In [5]:
%run scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x-duc.yaml --checkpoint pretrained_models/fast_421_res50-shuffle_256x192.pth --video ../output.mp4 --outdir ../output/ --qsize 50 --detbatch 5 --posebatch 50 --gpus 0 --pose_flow --save_video --showbox --vis_fast --debug

2023-10-02 23:43:57 [DEBUG]: Loaded backend agg version v2.2.


Load shuffle backbone...
Loading YOLO model..


C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pose model from pretrained_models/fast_421_res50-shuffle_256x192.pth...
[0]
cuda:0
Start pose tracking...



  1%|          | 93/15451 [03:13<8:53:12,  2.08s/it] 

===========================> Finish Model Running.
===========================> This video get 1355 frames in total.


Results have been written to json.ring remaining 0 images in the queue....


## Fast Pose DCN MSCOCO - Output seems to be missing frames

In [18]:
%run scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_2x-dcn.yaml --checkpoint pretrained_models/fast_dcn_res50_256x192.pth --video ../samplefencing.mp4 --outdir ../output/ --qsize 50 --detbatch 3 --posebatch 30 --gpus 0 --pose_track --save_video --vis_fast --debug

Loading YOLO model..
Loading pose model from pretrained_models/fast_dcn_res50_256x192.pth...
[0]
cuda:0
loading reid model from trackers/weights/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth...


100%|██████████| 14103/14103 [30:57<00:00,  7.59it/s]


===========================> Finish Model Running.
Results have been written to json.


## Fast Pose MultiDomain Missing frames also

In [5]:
%run scripts/demo_inference.py --cfg configs/halpe_coco_wholebody_136/resnet/256x192_res50_lr1e-3_2x-regression.yaml --checkpoint pretrained_models/multi_domain_fast50_regression_256x192.pth --video ../samplefencing.mp4 --outdir ../output/ --qsize 50 --detbatch 3 --posebatch 30 --gpus 0 --pose_track --save_video --vis_fast --debug

2023-09-25 09:03:24 [DEBUG]: Loaded backend agg version v2.2.


Loading YOLO model..


C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\anaconda3\envs\FYP\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pose model from pretrained_models/multi_domain_fast50_regression_256x192.pth...
[0]
cuda:0
loading reid model from trackers/weights/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth...


  0%|          | 0/14103 [00:00<?, ?it/s]C:\Users\admin\Documents\Anaconda Projects\AlphaPose\trackers\__init__.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  new_hm = torch.Tensor(new_hm).to(args.device)
100%|██████████| 14103/14103 [1:03:45<00:00,  3.69it/s]


===========================> Finish Model Running.
Results have been written to json.ring remaining 0 images in the queue...
